### Тема “Свёртки”

Берем отызывы за лето (из архива с материалами или предыдущего занятия)
1. Учим conv сеть для классификации
2. Рассмотреть 2-а варианта сеточек
- Инициализировать tf.keras.layers.Embedding предобученными векторами взять к примеру с https://rusvectores.org/ru/
- Инициализировать слой tf.keras.layers.Embedding по умолчанию (ну то есть вам ничего не делать с весами)

Сравнить две архитектуры с предобученными весами и когда tf.keras.layers.Embedding обучается сразу со всей сеточкой, что получилось лучше

In [1]:
!pip3 install stop_words pymorphy2 tensorflow scikit-learn nltk xlrd tensorflow-addons

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_addons as tfa

from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer
import re

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from keras.callbacks import TensorBoard 
from keras.metrics import categorical_crossentropy
from keras.callbacks import EarlyStopping

2022-11-14 11:26:06.404047: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
nltk.download("punkt")

[nltk_data] Downloading package punkt to /Users/dv/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [4]:
max_words = 300
max_len = 40
num_classes = 1


epochs = 20
batch_size = 512
print_batch_n = 100

In [5]:
df =  pd.read_excel('отзывы за лето.xls')

In [6]:
df.head()

,Rating,Content,Date
0,5,It just works!,2017-08-14
1,4,В целом удобноное приложение...из минусов хотя...,2017-08-14
2,5,Отлично все,2017-08-14
3,5,Стал зависать на 1% работы антивируса. Дальше ...,2017-08-14
4,5,"Очень удобно, работает быстро.",2017-08-14


In [7]:
sw = set(get_stop_words("ru"))
exclude = set(punctuation)
morpher = MorphAnalyzer()

def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()
    txt = re.sub("\sне", "не", txt)
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in sw]
    return " ".join(txt)

In [8]:
df.drop('Date', axis=1, inplace=True)
df_train, df_test = train_test_split(df, test_size=0.33, random_state=42)

In [9]:
df_train.head()

,Rating,Content
3517,5,Наконец-то исправили эту чушь с неоргинальной ...
12399,5,Удобно в использовании
3833,5,Отлично
13250,5,Класс
17941,5,Удобно


In [10]:
df_test.head()

,Rating,Content
1457,5,Всё отлично!!!
19916,5,Мне нравится. Удобно. Еще ни разу не подвел!
2112,3,Иногда слишком долго приходится ждать пока зап...
15154,5,Отличное приложение. Легко отслеживать свои ра...
18329,5,Все грамотно


In [11]:
df_train['Content'] = df_train['Content'].apply(preprocess_text)
df_test['Content'] = df_test['Content'].apply(preprocess_text)

In [12]:
train_corpus = " ".join(df_train["Content"])
train_corpus = train_corpus.lower()

In [13]:
tokens = word_tokenize(train_corpus)
tokens_filtered = [word for word in tokens if word.isalnum()]
dist = FreqDist(tokens_filtered)
tokens_filtered_top = [pair[0] for pair in dist.most_common(max_words-1)]
tokens_filtered_top[:10]

['приложение',
 'удобно',
 'работать',
 'удобный',
 'отлично',
 'нравиться',
 'хороший',
 'отличный',
 'телефон',
 'супер']

In [14]:
vocabulary = {v: k for k, v in dict(enumerate(tokens_filtered_top, 1)).items()}
len(vocabulary)

299

In [15]:
def text_to_sequence(text, maxlen):
    result = []
    tokens = word_tokenize(text.lower())
    tokens_filtered = [word for word in tokens if word.isalnum()]
    for word in tokens_filtered:
        if word in vocabulary:
            result.append(vocabulary[word])
    padding = [0]*(maxlen-len(result))
    return padding + result[-maxlen:]

In [16]:
x_train = np.asarray([text_to_sequence(text, max_len) for text in df_train["Content"]], dtype=np.int32)
x_test = np.asarray([text_to_sequence(text, max_len) for text in df_test["Content"]], dtype=np.int32)

In [17]:
le = LabelEncoder()
train_enc_labels = le.fit_transform(df_train['Rating']) 
test_enc_labels = le.transform(df_test['Rating'])
le.classes_

array([1, 2, 3, 4, 5])

In [18]:
num_classes = 5
y_train = tf.keras.utils.to_categorical(train_enc_labels, num_classes=num_classes)
y_test = tf.keras.utils.to_categorical(test_enc_labels, num_classes=num_classes)

In [19]:
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=128, input_length=max_len))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))
model.summary()

2022-11-14 11:26:27.622023: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 40, 128)           38400     
                                                                 
 conv1d (Conv1D)             (None, 38, 128)           49280     
                                                                 
 activation (Activation)     (None, 38, 128)           0         
                                                                 
 global_max_pooling1d (Globa  (None, 128)              0         
 lMaxPooling1D)                                                  
                                                                 
 dense (Dense)               (None, 10)                1290      
                                                                 
 activation_1 (Activation)   (None, 10)                0         
                                                        

In [20]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy', tfa.metrics.F1Score(num_classes=num_classes, average='macro')])

In [21]:
tensorboard = TensorBoard(log_dir='./logs', write_graph=True, write_images=True)

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard])

Epoch 1/20
25/25 [==============================] - 2s 54ms/step - loss: 1.2657 - accuracy: 0.6787 - f1_score: 0.1920 - val_loss: 0.9834 - val_accuracy: 0.7011 - val_f1_score: 0.1649
Epoch 2/20
25/25 [==============================] - 1s 43ms/step - loss: 0.9023 - accuracy: 0.7182 - f1_score: 0.2180 - val_loss: 0.8232 - val_accuracy: 0.7365 - val_f1_score: 0.2664
Epoch 3/20
25/25 [==============================] - 1s 43ms/step - loss: 0.7400 - accuracy: 0.7599 - f1_score: 0.2926 - val_loss: 0.7346 - val_accuracy: 0.7437 - val_f1_score: 0.2895
Epoch 4/20
25/25 [==============================] - 1s 42ms/step - loss: 0.6765 - accuracy: 0.7718 - f1_score: 0.3281 - val_loss: 0.7098 - val_accuracy: 0.7451 - val_f1_score: 0.3092
Epoch 5/20
25/25 [==============================] - 1s 42ms/step - loss: 0.6514 - accuracy: 0.7765 - f1_score: 0.3488 - val_loss: 0.6998 - val_accuracy: 0.7487 - val_f1_score: 0.3292
Epoch 6/20
25/25 [==============================] - 1s 43ms/step - loss: 0.6382 - acc

In [22]:
score = model.evaluate(x_test, y_test, batch_size=batch_size, verbose=1)

14/14 [==============================] - 0s 9ms/step - loss: 0.6900 - accuracy: 0.7633 - f1_score: 0.3610


!wget http://vectors.nlpl.eu/repository/20/220.zip

!unzip 220.zip

!pip install -U gensim

In [23]:
import gensim

In [24]:
word_model = gensim.models.KeyedVectors.load_word2vec_format('model.bin', binary=True)  

In [25]:
len(word_model)

249333

In [26]:
word_model_matrix = [word_model[i][:128] for i in range(300)]

In [27]:
initializer = tf.keras.initializers.Constant(word_model_matrix)

model_w = Sequential()
model_w.add(Embedding(input_dim=max_words, output_dim=128, embeddings_initializer =initializer, input_length=max_len))
model_w.add(Conv1D(128, 3))
model_w.add(Activation("relu"))
model_w.add(GlobalMaxPool1D())
model_w.add(Dense(10))
model_w.add(Activation("relu"))
model_w.add(Dense(num_classes))
model_w.add(Activation('softmax'))
model_w.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 40, 128)           38400     
                                                                 
 conv1d_1 (Conv1D)           (None, 38, 128)           49280     
                                                                 
 activation_3 (Activation)   (None, 38, 128)           0         
                                                                 
 global_max_pooling1d_1 (Glo  (None, 128)              0         
 balMaxPooling1D)                                                
                                                                 
 dense_2 (Dense)             (None, 10)                1290      
                                                                 
 activation_4 (Activation)   (None, 10)                0         
                                                      

In [28]:
model_w.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy', tfa.metrics.F1Score(num_classes=num_classes, average='macro')])

In [29]:
tensorboard = TensorBoard(log_dir='./logs', write_graph=True, write_images=True)

history_w = model_w.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard])

Epoch 1/20
25/25 [==============================] - 2s 52ms/step - loss: 1.3375 - accuracy: 0.6745 - f1_score: 0.2649 - val_loss: 0.8896 - val_accuracy: 0.7119 - val_f1_score: 0.2662
Epoch 2/20
25/25 [==============================] - 1s 43ms/step - loss: 0.8429 - accuracy: 0.7187 - f1_score: 0.2692 - val_loss: 0.8394 - val_accuracy: 0.7054 - val_f1_score: 0.2513
Epoch 3/20
25/25 [==============================] - 1s 43ms/step - loss: 0.7931 - accuracy: 0.7280 - f1_score: 0.2789 - val_loss: 0.8255 - val_accuracy: 0.7184 - val_f1_score: 0.2575
Epoch 4/20
25/25 [==============================] - 1s 43ms/step - loss: 0.7603 - accuracy: 0.7315 - f1_score: 0.2790 - val_loss: 0.8050 - val_accuracy: 0.7213 - val_f1_score: 0.2706
Epoch 5/20
25/25 [==============================] - 1s 43ms/step - loss: 0.7348 - accuracy: 0.7351 - f1_score: 0.2848 - val_loss: 0.7946 - val_accuracy: 0.7235 - val_f1_score: 0.2743
Epoch 6/20
25/25 [==============================] - 1s 43ms/step - loss: 0.7136 - acc

In [30]:
score_p = model_w.evaluate(x_test, y_test, batch_size=batch_size, verbose=1)

14/14 [==============================] - 0s 9ms/step - loss: 0.7780 - accuracy: 0.7495 - f1_score: 0.3144


In [31]:
score

[0.690024197101593, 0.7632737159729004, 0.36102837324142456]

In [32]:
score_p

[0.7779568433761597, 0.7494866251945496, 0.31437259912490845]

Предобученная модель показала результат ниже